<a href="https://colab.research.google.com/github/ManuelTorresM/Parallel_Programming/blob/main/profiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Use of Colab to profile some functions**

Code contributed by Samuele Moscatelli and Moreno Giussani

# **Notebook setup**

**Download the code**

In [ ]:
!git clone https://github.com/google/benchmark.git
!git clone https://github.com/google/googletest.git benchmark/googletest

Cloning into 'benchmark'...
remote: Enumerating objects: 8837, done.
remote: Counting objects: 100% (1437/1437), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 8837 (delta 1288), reused 1266 (delta 1195), pack-reused 7400 (from 1)
Receiving objects: 100% (8837/8837), 2.80 MiB | 10.56 MiB/s, done.
Resolving deltas: 100% (5951/5951), done.
Cloning into 'benchmark/googletest'...
remote: Enumerating objects: 27661, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 27661 (delta 0), reused 3 (delta 0), pack-reused 27651 (from 1)
Receiving objects: 100% (27661/27661), 12.84 MiB | 18.11 MiB/s, done.
Resolving deltas: 100% (20548/20548), done.


**Organize the code and install**

In [ ]:
!rm -rf benchmark/build
!cmake -E make_directory "benchmark/build"
!cmake -E chdir "benchmark/build" cmake -DCMAKE_BUILD_TYPE=Release ..
!cmake --build "benchmark/build" --config Release --target install

-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Failed to find LLVM FileCheck
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Google Benchmark version: v1.9.0-9-g23d8c1e5, normalized to 1.9.0.9
-- Looking for shm_open in rt
-- Looking for shm_open in rt - found
-- Performing Test HAVE_CXX_FLAG_WALL
-- Performing Test HAVE_CXX_FLAG_WALL - Success
-- Performing Test HAVE_CXX_FLAG_WEXTRA
-- Performing Test HAVE_CXX_FLAG_WEXTRA - Success
-- Performing Test HAVE_CXX_FLAG_WSHADOW
-- Performing Test HAVE_CXX_FLAG_WSHADOW - Success
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL
-- Performing Test HAVE_CXX_FLAG_WFLOAT_EQUAL - Success
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST
-- Performing Test HAVE_CXX_FLAG_WOLD_STYLE_CAST - Success
-- Performing Test HAVE_CXX_FLAG_WCONVE

# **Profiling a binary search**

In [ ]:
%%writefile binarysearch.cpp
#include <algorithm>
#include <stdlib.h>
#include <time.h>
#include <benchmark/benchmark.h>


void random_initialization(int* values,size_t len)
{
    for(size_t i=0;i<len;++i)
    {
        values[i] = rand() % (len*2);
    }
}

static void BM_BinarySearch(benchmark::State& state)
{
    int size=state.range(0);
    srand (time(NULL));

    int * nums= new int[size];
    random_initialization(nums,size);
    std::sort(nums,nums+size);

    for (auto _ :state)
        benchmark::DoNotOptimize(std::binary_search(nums,nums+size,size));
    state.SetComplexityN(state.range(0));
    delete nums;
}


BENCHMARK(BM_BinarySearch)
    ->RangeMultiplier(2)
    ->Range(1<<2, 1<<18)
    ->Complexity();


BENCHMARK_MAIN();

Writing binarysearch.cpp


In [ ]:
!g++ binarysearch.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram1

In [ ]:
!./testprogram1

2024-10-01T12:44:39+00:00
Running ./testprogram1
Run on (2 X 2200 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.70, 1.07, 0.58
-----------------------------------------------------------------
Benchmark                       Time             CPU   Iterations
-----------------------------------------------------------------
BM_BinarySearch/4            7.58 ns         7.25 ns     90657938
BM_BinarySearch/8            8.56 ns         8.19 ns     84204416
BM_BinarySearch/16           5.50 ns         5.47 ns    132121856
BM_BinarySearch/32           5.87 ns         5.85 ns    120985222
BM_BinarySearch/64           8.50 ns         8.45 ns     74630832
BM_BinarySearch/128          8.86 ns         8.84 ns     80764370
BM_BinarySearch/256          11.4 ns         11.4 ns     63017891
BM_BinarySearch/512          12.6 ns         12.5 ns     56617877
BM_BinarySearch/1024         11.4 ns         11.3

# **Profiling a power computation**

In [ ]:
%%writefile power_computation.cpp
#include <benchmark/benchmark.h>

long bpow(long  a,long n)
{   long half;
    if (n==0)
        return 1;
    if (n%2 == 0)
    {
        half=bpow(a,n/2);
        return half*half;
    }
    else
    {
        half=bpow(a,(n-1)/2);
        return half*half*a;
    }
}

static void BM_Pow(benchmark::State& state)
{
    long  exponent=state.range(0);
    const long  a=3;

    for (auto _ :state)
        bpow(a,exponent);
    state.SetComplexityN(state.range(0));
}


BENCHMARK(BM_Pow)
    ->RangeMultiplier(2)
    ->Range(1<<2, 1<<18)
    ->Complexity();


BENCHMARK_MAIN();


Writing power_computation.cpp


In [ ]:
!g++ power_computation.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram2

In [ ]:
!./testprogram2

2023-09-21T15:04:04+00:00
Running ./testprogram2
Run on (2 X 2200.21 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.22, 0.36, 0.32
--------------------------------------------------------
Benchmark              Time             CPU   Iterations
--------------------------------------------------------
BM_Pow/4            11.6 ns         11.3 ns     63326233
BM_Pow/8            10.5 ns         10.3 ns     52274698
BM_Pow/16           9.74 ns         9.67 ns     74029517
BM_Pow/32           12.1 ns         12.0 ns     60168691
BM_Pow/64           14.2 ns         14.2 ns     49990567
BM_Pow/128          15.9 ns         15.8 ns     44977059
BM_Pow/256          17.6 ns         17.5 ns     37468065
BM_Pow/512          22.3 ns         22.2 ns     30889645
BM_Pow/1024         25.3 ns         25.1 ns     28204743
BM_Pow/2048         27.9 ns         27.8 ns     25393073
BM_Pow/4096         30.6 ns    

# **Profiling a matrix computation**

In [ ]:
%%writefile strassen.cpp
#include <sstream>
#include <string>
#include <fstream>
#include <iostream>
#include <vector>
#include <math.h>
#include <benchmark/benchmark.h>

//source: https://github.com/wcochran/strassen_multiplier/blob/master/mm.c

////////////////////////////////////////////////

#include <stdlib.h>
#include <stdio.h>
//
// Classic O(N^3) square matrix multiplication.
// Z = X*Y
// All matrices are NxN and stored in row major order
// each with a specified pitch.
// The pitch is the distance (in double's) between
// elements at (row,col) and (row+1,col).
//
void mmult(int N,
           int Xpitch, const double X[],
           int Ypitch, const double Y[],
           int Zpitch, double Z[]) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++) {
      double sum = 0.0;
      for (int k = 0; k < N; k++)
        sum += X[i*Xpitch + k]*Y[k*Ypitch + j];
      Z[i*Zpitch + j] = sum;
    }
}

//
// S = X + Y
//
void madd(int N,
          int Xpitch, const double X[],
          int Ypitch, const double Y[],
          int Spitch, double S[]) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      S[i*Spitch + j] = X[i*Xpitch + j] + Y[i*Ypitch + j];
}

//
// S = X - Y
//
void msub(int N,
          int Xpitch, const double X[],
          int Ypitch, const double Y[],
          int Spitch, double S[]) {
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      S[i*Spitch + j] = X[i*Xpitch + j] - Y[i*Ypitch + j];
}

//
// Volker Strassen algorithm for matrix multiplication.
// Theoretical Runtime is O(N^2.807).
// Assume NxN matrices where N is a power of two.
// Algorithm:
//   Matrices X and Y are split into four smaller
//   (N/2)x(N/2) matrices as follows:
//          _    _          _   _
//     X = | A  B |    Y = | E F |
//         | C  D |        | G H |
//          -    -          -   -
//   Then we build the following 7 matrices (requiring
//   seven (N/2)x(N/2) matrix multiplications -- this is
//   where the 2.807 = log2(7) improvement comes from):
//     P0 = A*(F - H);
//     P1 = (A + B)*H
//     P2 = (C + D)*E
//     P3 = D*(G - E);
//     P4 = (A + D)*(E + H)
//     P5 = (B - D)*(G + H)
//     P6 = (A - C)*(E + F)
//   The final result is
//        _                                            _
//   Z = | (P3 + P4) + (P5 - P1)   P0 + P1              |
//       | P2 + P3                 (P0 + P4) - (P2 + P6)|
//        -                                            -
//
void mmult_strassen(int N,
                int Xpitch, const double X[],
                int Ypitch, const double Y[],
                int Zpitch, double Z[]) {
  //
  // Recursive base case.
  // If matrices are 16x16 or smaller we just use
  // the conventional algorithm.
  // At what size we should switch will vary based
  // on hardware platform.
  //
  if (N <= 16) {
    mmult(N, Xpitch, X, Ypitch, Y, Zpitch, Z);
    return;
  }

  const int n = N/2;      // size of sub-matrices

  const double *A = X;    // A-D matrices embedded in X
  const double *B = X + n;
  const double *C = X + n*Xpitch;
  const double *D = C + n;

  const double *E = Y;    // E-H matrices embeded in Y
  const double *F = Y + n;
  const double *G = Y + n*Ypitch;
  const double *H = G + n;

  double *P[7];   // allocate temp matrices off heap
  const int sz = n*n*sizeof(double);
  for (int i = 0; i < 7; i++)
    P[i] = (double *) malloc(sz);
  double *T = (double *) malloc(sz);
  double *U = (double *) malloc(sz);

  // P0 = A*(F - H);
  msub(n, Ypitch, F, Ypitch, H, n, T);
  mmult_strassen(n, Xpitch, A, n, T, n, P[0]);

  // P1 = (A + B)*H
  madd(n, Xpitch, A, Xpitch, B, n, T);
  mmult_strassen(n, n, T, Ypitch, H, n, P[1]);

  // P2 = (C + D)*E
  madd(n, Xpitch, C, Xpitch, D, n, T);
  mmult_strassen(n, n, T, Ypitch, E, n, P[2]);

  // P3 = D*(G - E);
  msub(n, Ypitch, G, Ypitch, E, n, T);
  mmult_strassen(n, Xpitch, D, n, T, n, P[3]);

  // P4 = (A + D)*(E + H)
  madd(n, Xpitch, A, Xpitch, D, n, T);
  madd(n, Ypitch, E, Ypitch, H, n, U);
  mmult_strassen(n, n, T, n, U, n, P[4]);

  // P5 = (B - D)*(G + H)
  msub(n, Xpitch, B, Xpitch, D, n, T);
  madd(n, Ypitch, G, Ypitch, H, n, U);
  mmult_strassen(n, n, T, n, U, n, P[5]);

  // P6 = (A - C)*(E + F)
  msub(n, Xpitch, A, Xpitch, C, n, T);
  madd(n, Ypitch, E, Ypitch, F, n, U);
  mmult_strassen(n, n, T, n, U, n, P[6]);

  // Z upper left = (P3 + P4) + (P5 - P1)
  madd(n, n, P[4], n, P[3], n, T);
  msub(n, n, P[5], n, P[1], n, U);
  madd(n, n, T, n, U, Zpitch, Z);

  // Z lower left = P2 + P3
  madd(n, n, P[2], n, P[3], Zpitch, Z + n*Zpitch);

  // Z upper right = P0 + P1
  madd(n, n, P[0], n, P[1], Zpitch, Z + n);

  // Z lower right = (P0 + P4) - (P2 + P6)
  madd(n, n, P[0], n, P[4], n, T);
  madd(n, n, P[2], n, P[6], n, U);
  msub(n, n, T, n, U, Zpitch, Z + n*(Zpitch + 1));

  free(U);  // deallocate temp matrices
  free(T);
  for (int i = 6; i >= 0; i--)
    free(P[i]);
}

void mprint(int N, int pitch, const double M[]) {
  for (int i = 0; i < N; i++) {
    for (int j = 0; j < N; j++)
      printf("%+0.4f ", M[i*pitch + j]);
    printf("\n");
  }
}


void mrand(int N, int pitch, double M[]) {
  const double r = 10.0;
  for (int i = 0; i < N; i++)
    for (int j = 0; j < N; j++)
      M[i*pitch + j] = r*(2*drand48() - 1);
}

#include <sys/time.h>

int timeval_subtract(struct timeval *result,
                     struct timeval *t2, struct timeval *t1) {
  long int diff =
    (t2->tv_usec + 1000000 * t2->tv_sec) -
    (t1->tv_usec + 1000000 * t1->tv_sec);
  result->tv_sec = diff / 1000000;
  result->tv_usec = diff % 1000000;
  return (diff<0);
}

void timeval_print(struct timeval *tv) {
  char buffer[30];
  time_t curtime;

  printf("%ld.%06ld", (long int) tv->tv_sec, (long int) tv->tv_usec);
  curtime = tv->tv_sec;
  strftime(buffer, 30, "%m-%d-%Y  %T", localtime(&curtime));
  printf(" = %s.%06ld\n", buffer, (long int) tv->tv_usec);
}

////////////////////////////////////////////////


static void BM_Strassen (benchmark::State& state) {
    int N =state.range(0);
    double *A, *B, *C, *Cstrassen;
    A = (double*) malloc(N*N*sizeof(double));
    B = (double*) malloc(N*N*sizeof(double));
    C = (double*) malloc(N*N*sizeof(double));
    mrand(N, N, A);
    mrand(N, N, B);
    mrand(N, N, C);

    for (auto _: state)
        mmult_strassen(N, N, A, N, B, N, C);
    state.SetComplexityN(N);
}


static void BM_ClassicalMultiplication (benchmark::State& state) {
    int N =state.range(0);
    double *A, *B, *C, *Cstrassen;
    A = (double*) malloc(N*N*sizeof(double));
    B = (double*) malloc(N*N*sizeof(double));
    C = (double*) malloc(N*N*sizeof(double));
    mrand(N, N, A);
    mrand(N, N, B);
    mrand(N, N, C);

    for (auto _: state)
        mmult(N, N, A, N, B, N, C);
    state.SetComplexityN(N);
}


BENCHMARK(BM_Strassen)
    ->RangeMultiplier(2)
    ->Range(2,2048)
    ->Complexity();

BENCHMARK(BM_ClassicalMultiplication)
    ->RangeMultiplier(2)
    ->Range(2,1024)
    ->Complexity();
BENCHMARK_MAIN();



Overwriting strassen.cpp


In [ ]:
!g++ strassen.cpp -O2 -std=c++11 -isystem benchmark/include -Lbenchmark/build/src -lbenchmark -lpthread -o testprogram3

In [ ]:
!./testprogram3

2023-09-21T15:07:13+00:00
Running ./testprogram3
Run on (2 X 2200.21 MHz CPU s)
CPU Caches:
  L1 Data 32 KiB (x1)
  L1 Instruction 32 KiB (x1)
  L2 Unified 256 KiB (x1)
  L3 Unified 56320 KiB (x1)
Load Average: 0.19, 0.29, 0.30
--------------------------------------------------------------------------
Benchmark                                Time             CPU   Iterations
--------------------------------------------------------------------------
BM_Strassen/2                         20.6 ns         20.5 ns     34376187
BM_Strassen/4                         80.4 ns         80.1 ns      8818078
BM_Strassen/8                          508 ns          504 ns      1000000
BM_Strassen/16                        3686 ns         3665 ns       195634
BM_Strassen/32                       30156 ns        30035 ns        21969
BM_Strassen/64                      230933 ns       230159 ns         3128
BM_Strassen/128                    1824632 ns      1819525 ns          396
BM_Strassen/256       